In [79]:
# !pip install requests-html requests pandas

In [78]:
import requests
from requests_html import HTML
import time
import pandas as pd

In [4]:
base_url = 'https://stackoverflow.com/questions/tagged/'
tag = "python"
query_filter = "Votes"
url = f"{base_url}{tag}?tab={query_filter}"
url

'https://stackoverflow.com/questions/tagged/python?tab=Votes'

In [7]:
r = requests.get(url)
html_str = r.text
html = HTML(html=html_str)

In [8]:
#  = HTML(html=html_str)

In [9]:
question_summaries = html.find(".question-summary") # selenium
print(question_summaries)

[<Element 'div' class=('question-summary',) id='question-summary-231767'>, <Element 'div' class=('question-summary',) id='question-summary-419163'>, <Element 'div' class=('question-summary',) id='question-summary-394809'>, <Element 'div' class=('question-summary',) id='question-summary-100003'>, <Element 'div' class=('question-summary',) id='question-summary-82831'>, <Element 'div' class=('question-summary',) id='question-summary-89228'>, <Element 'div' class=('question-summary',) id='question-summary-38987'>, <Element 'div' class=('question-summary',) id='question-summary-273192'>, <Element 'div' class=('question-summary',) id='question-summary-522563'>, <Element 'div' class=('question-summary',) id='question-summary-136097'>, <Element 'div' class=('question-summary',) id='question-summary-3437059'>, <Element 'div' class=('question-summary',) id='question-summary-3207219'>, <Element 'div' class=('question-summary',) id='question-summary-952914'>, <Element 'div' class=('question-summar

In [14]:
question_summaries[0].text

"10289\nvotes\n41answers\n2.3m views\nWhat does the “yield” keyword do?\nWhat is the use of the yield keyword in Python, and what does it do? For example, I'm trying to understand this code1: def _get_child_candidates(self, distance, min_dist, max_dist): if self....\npython iterator generator yield coroutine\nasked Oct 23 '08 at 22:21\nAlex. S.\n117k1515 gold badges4949 silver badges6161 bronze badges"

In [33]:
columns = ['votes', 'vote_title', 'num_answers', 'views', 'question', 'short_desc', 'tags', 'date', 'user', 'user_details']

In [37]:
# this_row = list(question_summaries[0].text.split("\n"))
# this_row

In [36]:
# len(this_row) == len(columns)

In [34]:
# row_data = dict(zip(columns, this_row))
# row_data

In [35]:
# for column, row_v in zip(columns, this_row):
#     print(column, row_v)

In [52]:
key_names = ['question', 'votes', 'tags']
classes_needed = ['.question-hyperlink', '.vote', '.tags']
this_question_element = question_summaries[0]
this_question_element.find('.question-hyperlink', first=True).text
this_question_element.find('.vote', first=True).text.replace('\nvotes', '')

'10289'

In [56]:
def clean_scraped_data(text, keyname=None):
    if keyname == 'votes':
        return text.replace('\nvotes', '')
    return text

In [57]:
datas = []

for q_el in question_summaries:
    question_data = {}
    for i, _class in enumerate(classes_needed):
        sub_el = q_el.find(_class, first=True)
        keyname = key_names[i]
        question_data[keyname] = clean_scraped_data(sub_el.text, keyname=keyname)
    datas.append(question_data)

In [58]:
# datas[0]

{'question': 'What does the “yield” keyword do?',
 'votes': '10289',
 'tags': 'python iterator generator yield coroutine'}

In [63]:
def parse_tagged_page(html):
    question_summaries = html.find(".question-summary")
    key_names = ['question', 'votes', 'tags']
    classes_needed = ['.question-hyperlink', '.vote', '.tags']
    datas = []
    for q_el in question_summaries:
        question_data = {}
        for i, _class in enumerate(classes_needed):
            sub_el = q_el.find(_class, first=True)
            keyname = key_names[i]
            question_data[keyname] = clean_scraped_data(sub_el.text, keyname=keyname)
        datas.append(question_data)
    return datas

In [64]:
def extract_data_from_url(url):
    r = requests.get(url)
    if r.status_code not in range(200, 299):
        return []
    html_str = r.text
    html = HTML(html=html_str)
    datas = parse_tagged_page(html)
    return datas

In [70]:
def scrape_tag(tag = "python", query_filter = "Votes", max_pages=50, pagesize=25):
    base_url = 'https://stackoverflow.com/questions/tagged/'
    datas = []
    for p in range(max_pages):
        page_num = p + 1
        url = f"{base_url}{tag}?tab={query_filter}&page={page_num}&pagesize={pagesize}"
        datas += extract_data_from_url(url)
        time.sleep(1.2)
    return datas

In [72]:
datas = scrape_tag(tag='python')

In [76]:
len(datas)

2500

In [80]:
df = pd.DataFrame(datas)
df.head()

,question,votes,tags
0,What does the “yield” keyword do?,10289,python iterator generator yield coroutine
1,What does if __name__ == “__main__”: do?,6134,python namespaces main python-module idioms
2,Does Python have a ternary conditional operator?,6104,python operators ternary-operator conditional-...
3,What are metaclasses in Python?,5752,python oop metaclass python-datamodel
4,How do I check whether a file exists without e...,5653,python file file-exists


In [81]:
df.shape

(2500, 3)

In [82]:
df.to_csv("python.csv", index=False)

In [ ]:
# auto-tagging question bot